In [7]:
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm

In [2]:
file_path = '../pyanomaly-master/output/merge.pickle'
with open(file_path, 'rb') as f:
    obj = pkl.load(f)
    
df = pd.DataFrame(obj)

In [3]:
# Get indexes
dates = df.index.get_level_values('date')
permno_ids = df.index.get_level_values('permno')
dates[0:10]

DatetimeIndex(['2010-01-31', '2010-02-28', '2010-03-31', '2010-04-30',
               '2010-05-31', '2010-06-30', '2010-07-31', '2010-08-31',
               '2010-09-30', '2010-10-31'],
              dtype='datetime64[ns]', name='date', freq=None)

In [4]:
# Count unique permnos
count = 0
permno_set = set()
for p in permno_ids.to_list():
    if p not in permno_set:
        count += 1
        permno_set.add(p)
count

9206

In [5]:
exclude_columns = ["gvkey", "datadate", "primary", "exchcd", "ret", "exret", "rf"]
exret = df.exret
features = df.drop(columns=exclude_columns)
exret.head()

date        permno
2010-01-31  10001    -0.018960
2010-02-28  10001    -0.000666
2010-03-31  10001     0.020575
2010-04-30  10001     0.124274
2010-05-31  10001     0.004717
Name: exret, dtype: float64

In [ ]:
# Build data for lstm
past = 5 # 5 months of data
future = 1 # next month risk prem
X_arr,y = [],[]

for p in tqdm(permno_set):
    idxs = [idx for idx, val in enumerate(permno_ids) if val == p]
    temp_df = df.iloc[idxs]
    temp_exret = exret.iloc[idxs]
    
    for i in range(len(temp_df) - past - future + 1):
        X_arr.append(temp_df.iloc[i:i+past])
        y.append(temp_exret.iloc[i+past:i+past+future])
        

 38%|██████████████████████████████████████████████████████▉                                                                                           | 3464/9206 [41:41<15:48,  6.05it/s]